<a href="https://colab.research.google.com/github/namyaagrawal03/Insurance_fraud/blob/main/interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install typing-extensions==4.4.0

  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.108.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.4.0 which is incompatible.
openai 1.6.1 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.4.0 which is incompatible.
pydantic 2.5.3 requires typing-extensions>=4.6.1, but you have typing-extensions 4.4.0 which is incompatible.
pydantic-core 2.14.6 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.4.0 which is incompatible.


In [14]:
!pip install --upgrade lida -q
!pip install --upgrade fastapi -q
!pip install --upgrade pydantic -q
!pip install --upgrade pydantic-core -q
!pip install --upgrade tensorflow-probability -q

In [15]:
!pip install gradio -q

In [16]:
import gradio as gr

In [17]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')
rfc = joblib.load('/content/drive/MyDrive/Colab Notebooks/trained_model.pkl')
scaler = StandardScaler()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
def predict_fraud(Make, AccidentArea, Sex, Age, MaritalStatus, PolicyType,
       VehiclePrice, Deductible, DriverRating, PastNumberOfClaims,
       AgeOfVehicle, PoliceReportFiled, WitnessPresent, AgentType,
       NumberOfCars):
    # Assuming x_test is a DataFrame or array with the same features as your model
    test_data = [[Make, AccidentArea, Sex, Age, MaritalStatus, PolicyType,
       VehiclePrice, Deductible, DriverRating, PastNumberOfClaims,
       AgeOfVehicle, PoliceReportFiled, WitnessPresent, AgentType,
       NumberOfCars]]  # Create test data

    categorical_cols = test_data.select_dtypes(include =['object'])
    numerical_col = test_data.select_dtypes(include= ['int64'])

    #one hot encoding
    categorical_cols= pd.get_dummies(categorical_cols, drop_first= True)

    #scaling
    numerical_col=scaler.fit_transform(numerical_col)
    test_data=pd.concat([numerical_col, categorical_cols], axis=1)

    # Make predictions using the loaded RandomForestClassifier model
    predicted_label = rfc.predict(test_data)

    if predicted_label[0] == 1:
        output_label = "Fraud Detected"
    else:
        output_label = "No Fraud Detected"

    return output_label

In [12]:
make_input = gr.inputs.Number(label="Make of the vehicle")
accident_area_input = gr.inputs.Textbox(label="Accident Area")
sex_input = gr.inputs.Number(label="Sex")
age_input= gr.inputs.Number(label="Age")
marital_status_input= gr.inputs.Number(label="Marital Status")
policy_type_input= gr.inputs.Number(label="Policy Type")
vehicle_price_input= gr.inputs.Number(label="Price of the Vehicle")
deductible_input= gr.inputs.Number(label="Deductible Amount")
driver_rating_input= gr.inputs.Number(label="Driver Rating")
past_num_of_claims=gr.inputs.Number(label= "Past Number of Claims")
policy_type_input= gr.inputs.Number(label="Policy Type")
age_of_vehicle= gr.inputs.Number(label="Age of the Vehicle")
police_report_filed= gr.inputs.Number(label="Was a police report filed?")
witness_present= gr.inputs.Number(label="Was any witness present?")


AttributeError: ignored